In [ ]:
import pandas as pd
import numpy as np

The experimental results of Learning-to-stack can be found in "Learning-to-stack_meta learner"

The following are the experimental results of the comparison model

# Reading Data

In [ ]:
file_path = r''
df_all = pd.read_excel(file_path)

# Comparison Model

In [ ]:
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report

In [ ]:
# 假设 df_all 包含特征和标签
X = df_all.drop('健康状况', axis=1)
y = df_all['健康状况']

## Stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [ ]:
# 定义基学习器
base_estimators = [
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
    ('cat', CatBoostClassifier(silent=True)),
    ('lgb', LGBMClassifier())
]

In [ ]:
# 定义元学习器
meta_estimator = LogisticRegression()

# 定义Stacking模型
stacking_model = StackingClassifier(estimators=base_estimators, final_estimator=meta_estimator, cv=3)

In [ ]:
# 定义参数范围进行随机搜索
param_distributions = {
    'xgb__n_estimators': [100, 200, 300],
    'xgb__max_depth': [3, 6, 9],
    'xgb__learning_rate': [0.01, 0.1, 0.2],
    'cat__iterations': [100, 200, 300],
    'cat__depth': [4, 6, 8],
    'cat__learning_rate': [0.01, 0.1, 0.2],
    'lgb__n_estimators': [100, 200, 300],
    'lgb__num_leaves': [31, 50, 70],
    'lgb__learning_rate': [0.01, 0.1, 0.2]
}

In [ ]:
# 定义三折交叉验证
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 使用随机搜索进行参数调优
random_search = RandomizedSearchCV(estimator=stacking_model, param_distributions=param_distributions, n_iter=50, cv=cv, scoring='roc_auc', random_state=42)
random_search.fit(X, y)

In [ ]:
# 获取最佳模型和参数
best_stacking_model = random_search.best_estimator_

In [ ]:
# 通过三折交叉验证输出拟合结果
y_pred = cross_val_predict(best_stacking_model, X, y, cv=cv)
y_prob = cross_val_predict(best_stacking_model, X, y, cv=cv, method='predict_proba')[:, 1]

# 计算评价指标
auc = roc_auc_score(y, y_prob)
accuracy = accuracy_score(y, y_pred)
f1 = f1_score(y, y_pred)

# 输出评价指标和报告
print("Stacking")
print("AUC:", auc)
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y, y_pred))

## XGB

In [ ]:
from xgboost import XGBClassifier

In [ ]:
# 定义XGBoost模型
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# 定义参数范围进行随机搜索
param_distributions = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2]
}

In [ ]:
# 定义三折交叉验证
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# 使用随机搜索进行参数调优
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_distributions, n_iter=50, cv=cv, scoring='roc_auc', random_state=42)
random_search.fit(X, y)

In [ ]:
# 获取最佳模型和参数
best_xgb_model = random_search.best_estimator_

In [ ]:
# 通过三折交叉验证输出拟合结果
y_pred = cross_val_predict(best_xgb_model, X, y, cv=cv)
y_prob = cross_val_predict(best_xgb_model, X, y, cv=cv, method='predict_proba')[:, 1]

# 计算评价指标
auc = roc_auc_score(y, y_prob)
accuracy = accuracy_score(y, y_pred)
f1 = f1_score(y, y_pred)

# 输出评价指标和报告
print("XGBoost")
print("AUC:", auc)
print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y, y_pred))

## CAT

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
# 定义CatBoost模型
catboost_model = CatBoostClassifier(silent=True)

# 定义参数范围进行随机搜索
param_distributions = {
    'iterations': [100, 200, 300],
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.1, 0.2],
    'l2_leaf_reg': [1, 3, 5, 7],
    'bagging_temperature': [0.1, 0.5, 1.0],
    'border_count': [32, 50, 100]
}

# 定义三折交叉验证
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [ ]:
# 使用随机搜索进行参数调优
random_search = RandomizedSearchCV(estimator=catboost_model, param_distributions=param_distributions, n_iter=50, cv=cv, scoring='roc_auc', random_state=42)
random_search.fit(X, y)

In [ ]:
# 获取最佳模型和参数
best_catboost_model = random_search.best_estimator_

In [ ]:
# 通过三折交叉验证输出拟合结果
y_pred = cross_val_predict(best_catboost_model, X, y, cv=cv)
y_prob = cross_val_predict(best_catboost_model, X, y, cv=cv, method='predict_proba')[:, 1]

# 计算评价指标
auc = roc_auc_score(y, y_prob)
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

# 输出评价指标和报告
print("Catboost")
print("AUC:", auc)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y, y_pred))

## LGBM

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
# 定义LightGBM模型
lgbm_model = LGBMClassifier()

# 定义参数范围进行随机搜索
param_distributions = {
    'n_estimators': [100, 200, 300],
    'num_leaves': [31, 50, 70],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'reg_alpha': [0, 0.01, 0.1],
    'reg_lambda': [1, 1.5, 2]
}

# 定义三折交叉验证
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [ ]:
# 使用随机搜索进行参数调优
random_search = RandomizedSearchCV(estimator=lgbm_model, param_distributions=param_distributions, n_iter=50, cv=cv, scoring='roc_auc', random_state=42)
random_search.fit(X, y)

In [ ]:
# 获取最佳模型和参数
best_lgbm_model = random_search.best_estimator_

In [ ]:
# 通过三折交叉验证输出拟合结果
y_pred = cross_val_predict(best_lgbm_model, X, y, cv=cv)
y_prob = cross_val_predict(best_lgbm_model, X, y, cv=cv, method='predict_proba')[:, 1]

# 计算评价指标
auc = roc_auc_score(y, y_prob)
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

# 输出评价指标和报告
print("LightGBM")
print("AUC:", auc)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Classification Report:\n", classification_report(y, y_pred))